<a href="https://colab.research.google.com/github/omeryanai/omeryanai/blob/main/Copy_of_BGU_DL_ass1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

Python 3.9.16


In [18]:
import numpy as np
import pickle

a.	initialize_parameters(layer_dims)

input: an array of the dimensions of each layer in the network (layer 0 is the size of the flattened input, layer L is the output softmax) <br>

Example with input layer 784 neurons, output layer 10 neurons:  layer_dims = [784,30,20,10]<br>

output: a dictionary containing the initialized W and b parameters of each layer (W1…WL, b1…bL).

Hint: Use the randn and zeros functions of numpy to initialize W and b, respectively


In [2]:
def initialize_parameters(layer_dims,verbose=False):
  parameters = {}
  num_layers=layer_dims.shape[0]


  for current_layer in range(1,num_layers):
    prev_layer_size=layer_dims[current_layer-1]
    curr_layer_size=layer_dims[current_layer]
    w=np.random.randn(curr_layer_size,prev_layer_size)*0.01
    B=np.zeros((curr_layer_size,1))*0.01
    parameters['W'+str(current_layer)]=w
    parameters['B'+str(current_layer)]=B
    if verbose:
      print('layer ',current_layer,' W ',w.shape,' B ',B.shape)

  return parameters

b.	linear_forward(A, W, b)

Description: Implement the linear part of a layer's forward propagation.

input: 
A – the activations of the previous layer
W – the weight matrix of the current layer (of shape [size of current layer, size of previous layer])
B – the bias vector of the current layer (of shape [size of current layer, 1])

Output:
Z – the linear component of the activation function (i.e., the value before applying the non-linear function)
linear_cache – a dictionary containing A, W, b (stored for making the backpropagation easier to compute)


In [55]:
def linear_forward(A_prev, W, b,verbose=False):
  if verbose:
    print('A_prev',A_prev.shape, 'W.T ',W.T.shape,' B ',b.shape)
 # Z=np.add(b,np.matmul(A,W.T))

  Z = np.dot(W, A_prev) + b
  if verbose:
    print('W ',W.shape,'A_prev ',A_prev.shape, ' B ',b.shape,'Z',Z.shape)
  linear_cache= {'A_prev':A_prev,'W':W,'B':b,'Z':Z}
  return Z,linear_cache


c.	softmax(Z)

Input:
Z – the linear component of the activation function

Output:
A – the activations of the layer
activation_cache – returns Z, which will be useful for the backpropagation

note:
Softmax can be thought of as a sigmoid for multi-class problems. The formula for softmax for each node in the output layer is as follows:
softmax = exp(zi) / Sum(exp(zi))

In [6]:
def softmax(Z):
  A= (np.exp(Z)/np.exp(Z).sum())
#  activation_cache={'Z':Z}
  activation_cache={'A':A}
  return A,activation_cache     # A is the softmax result  ,  activation cache with Z is  for backprop 

d.	relu(Z)
Input:
Z – the linear component of the activation function

Output:
A – the activations of the layer
activation_cache – returns Z, which will be useful for the backpropagation


In [7]:
def relu(Z):
  A=np.maximum(0,Z)
# activation_cache={'Z':Z}
  activation_cache={'A':A}
  return A,activation_cache     # A is the softmax result  ,  activation cache with Z is  for backprop 

e.	linear_activation_forward(A_prev, W, B, activation)
Description:
Implement the forward propagation for the LINEAR->ACTIVATION layer

Input:
A_prev – activations of the previous layer
W – the weights matrix of the current layer
B – the bias vector of the current layer
Activation – the activation function to be used (a string, either “softmax” or “relu”)

Output:
A – the activations of the current layer
cache – a joint dictionary containing both linear_cache and activation_cache


In [8]:
def linear_activation_forward(A_prev, W, B, activation,verbose=False):
  linear_cache={}
  linear_activation={}
  Z,linear_cache=linear_forward(A_prev,W,B)
  if activation=='softmax':
    A,activation_cache=softmax(Z)
  else:   # relu
    A,activation_cache=relu(Z)
  if verbose:
    print('linear_activation_forward Z',A_prev.shape)
    print('linear_activation_forward A',A.shape)
    #print(linear_cache)
    #print(activation_cache)
  cache=linear_cache | activation_cache 
  return A,cache


f.	L_model_forward(X, parameters, use_batchnorm)
Description:
Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SOFTMAX computation

Input:
X – the data, numpy array of shape (input size, number of examples)
parameters – the initialized W and b parameters of each layer
use_batchnorm - a boolean flag used to determine whether to apply batchnorm after the activation (note that this option needs to be set to “false” in Section 3 and “true” in Section 4).

Output:
AL – the last post-activation value
caches – a list of all the cache objects generated by the linear_forward function


In [9]:
def  L_model_forward(X, parameters, number_of_layers,use_batchnorm=False,verbose=False):
 
    if not(use_batchnorm):
        A=X  # for layer 0 
        caches=[]
        #caches.append(['Inputs'+str(0),A])

        for l in range(1,number_of_layers-1):
            A_prev=A
            if verbose:
              print('layer ',l,' A ',A.shape,A)
              print(parameters['W'+str(l)].shape,parameters['B'+str(l)].shape)
            A,c = linear_activation_forward(A_prev,parameters['W'+str(l)],parameters['B'+str(l)],'relu')
            caches.append(['Layer'+str(l),c])

        l=l+1   # for last layer  
        AL,c= linear_activation_forward(A,parameters['W'+str(l)],parameters['B'+str(l)],'softmax')
        caches.append(['Layer'+str(l),c])
    return AL,caches

g.	compute_cost(AL, Y)
Description:
Implement the cost function defined by equation. The requested cost function is categorical cross-entropy loss. The formula is as follows 

-1/m * sum( y*log(Ypred) )

Input:
AL – probability vector corresponding to your label predictions, shape (num_of_classes, number of examples)
Y – the labels vector (i.e. the ground truth)
Output:
cost – the cross-entropy cost


In [53]:
def compute_cost(AL, Y,verbose=False):
  m=Y.shape[1]
  J=-(1/m)*np.multiply(Y,np.log(AL)).sum()
  if verbose:
    print('J is ',J)
    print('AL ',AL.shape,' log ',np.log(AL).shape,'Y',Y.shape)
    print('cost is',J,'cost shape',J.shape)
  return J


a.	Linear_backward(dZ, cache)
description:
Implements the linear part of the backward propagation process for a single layer

Input:
dZ – the gradient of the cost with respect to the linear output of the current layer (layer l)
cache – tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

Output:
dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
dW -- Gradient of the cost with respect to W (current layer l), same shape as W
db -- Gradient of the cost with respect to b (current layer l), same shape as b


In [11]:
def Linear_backward(dZ, cache,verbose=False):
  A_prev=cache['A_prev']
  W=cache['W']
  B=cache['B']
  
  m=A_prev.shape[1]

  dW=1/m*np.matmul(dZ,A_prev.T)
  dA_prev=np.dot(W.T, dZ)   ##### not clear how to compute dA   #### this code from https://datascience-enthusiast.com/DL/Building-your-Deep-Neural-Network-Step-by-Step.html
  dB=1/m*np.mean(dZ,axis=1,keepdims=True)
  
  if verbose:
    print('dZ',dZ.shape,'A_prev',A_prev.shape,'W',W.shape,'B',B.shape,'dA_prev',dA_prev.shape,'dW',dW.shape,'dB',dB.shape)
  return dA_prev, dW, dB

b.	linear_activation_backward(dA, cache, activation)
Description:
Implements the backward propagation for the LINEAR->ACTIVATION layer. The function first computes dZ and then applies the linear_backward function.

Some comments:
The derivative of ReLU is 1 if x>0 , and 0 otherwise.

The derivative of the softmax function is: p_i-y_i, where p_i is the softmax-adjusted probability of the class and y_i is the “ground truth” (i.e. 1 for the real class, 0 for all others) 
You should use the activations cache created earlier for the calculation of the activation derivative and the linear cache should be fed to the linear_backward function

Input:
dA – post activation gradient of the current layer
cache – contains both the linear cache and the activations cache

Output:
dA_prev – Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
dW – Gradient of the cost with respect to W (current layer l), same shape as W
db – Gradient of the cost with respect to b (current layer l), same shape as b


In [12]:
def linear_activation_backward(dA, Y, cache, activation,verbose=False):
  if activation=='relu':
    dZ=relu_backward(dA,cache['Z'])
    #print('relu derivative from dA to dZ ???')
  if activation=='softmax':
    dZ=softmax_backward(dA,Y, cache['A'])
  
  dA_prev, dW, dB = Linear_backward(dZ,cache,verbose=False)
  if verbose:
    print ('dZ ',dZ.shape )
    print('dA_prev ',dA_prev)
    print('cache ',cache)

  return dA_prev, dW, dB

d.	softmax_backward (dA, activation_cache)
Description:
Implements backward propagation for a softmax unit

Input:
dA – the post-activation gradient
activation_cache – contains Z (stored during the forward propagation)

Output:
dZ – gradient of the cost with respect to Z


In [13]:
def softmax_backward(dA, Y ,activation_cache):

  A= activation_cache
  dZ=A-Y


 # y = da - a / ( -(2a+1 ))
  return dZ

c.	relu_backward (dA, activation_cache)
Description:
Implements backward propagation for a ReLU unit

Input:
dA – the post-activation gradient
activation_cache – contains Z (stored during the forward propagation)

Output:
dZ – gradient of the cost with respect to Z


In [14]:
def relu_backward (dA, activation_cache):
  A = activation_cache
  dZ = np.array(A, copy=True)
  dZ[A <= 0] = 0
  #dZ=np.maximum(0,dA)
  return dZ

e.	L_model_backward(AL, Y, caches)
Description:
Implement the backward propagation process for the entire network.

Some comments:
the backpropagation for the softmax function should be done only once as only the output layers uses it and the RELU should be done iteratively over all the remaining layers of the network. 

Input:
AL - the probabilities vector, the output of the forward propagation (L_model_forward)
Y - the true labels vector (the "ground truth" - true classifications)
Caches - list of caches containing for each layer: a) the linear cache; b) the activation cache

Output:
Grads - a dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ...


In [15]:
def L_model_backward(AL, Y, caches,number_of_layers,verbose=False):
      
      grads={}

      last_layer=number_of_layers-1  # for last layer
      #print(AL.shape,Y.shape)
      
      dA = -np.divide(Y,AL)+(np.divide(1-Y,1-AL))

      #dZ=AL-Y
      l=last_layer
      cache=caches[l-1][1]   # select cache of last layer
 ###############
      #print('start backprop of layer' , l)
      dA_prev, dW, dB=linear_activation_backward(dA, Y, cache,'softmax',verbose=False)
      if verbose:
              print('Received dA_prev',dA_prev.shape,'dW',dW.shape,'dB',dB.shape)

      grads["dA" + str(l)] = dA_prev 
      grads["dW" + str(l)] = dW
      grads["dB" + str(l)] = dB

      for l in range(last_layer-1,0,-1):
            cache=caches[l-1][1]

            if verbose:
              print('layer ',l,' cache ',cache)
              print('sending dA to lin_act_bck ',dA.shape)
              print('dA',dA)
              print('start backprop of layer' , l)
            dA_prev, dW, dB=linear_activation_backward(dA, Y, cache,'relu',verbose=False)

            grads["dA" + str(l)] = dA_prev 
            grads["dW" + str(l)] = dW
            grads["dB" + str(l)] = dB
            if verbose:
              print('dA',dA_prev.shape,'dW',dW.shape,'dB',dB.shape)
         
      return grads

f.	Update_parameters(parameters, grads, learning_rate)
Description:
Updates parameters using gradient descent

Input:
parameters – a python dictionary containing the DNN architecture’s parameters
grads – a python dictionary containing the gradients (generated by L_model_backward)
learning_rate – the learning rate used to update the parameters (the “alpha”)

Output:
parameters – the updated values of the parameters object provided as input


In [16]:
def update_parameters(parameters, grads, learning_rate,number_of_layers,verbose=False):
   for l in range(number_of_layers-1,0,-1):    # there are 4 layers, so last layer is 3 
        if verbose:
          print(parameters)
        W=parameters['W'+str(l)]
        B=parameters['B'+str(l)]
        dW=grads['dW'+str(l)]
        dB=grads['dB'+str(l)]
        if verbose:
              print('layer ',l)
              print('w',parameters['W'+str(l)].shape)
              print('B',parameters['B'+str(l)].shape)
              print('dW',dW.shape)
              print('dB',dB.shape)
              
        new_W=W-learning_rate*dW
        parameters['W'+str(l)]=new_W

        new_B=B-learning_rate*dB
        parameters['B'+str(l)]=new_B

   return parameters


a.	L_layer_model(X, Y, layers_dims, learning_rate, num_iterations, batch_size)
Description:
Implements a L-layer neural network. All layers but the last should have the ReLU activation function, and the final layer will apply the softmax activation function. The size of the output layer should be equal to the number of labels in the data. Please select a batch size that enables your code to run well (i.e. no memory overflows while still running relatively fast).

Hint: the function should use the earlier functions in the following order: initialize -> L_model_forward -> compute_cost -> L_model_backward -> update parameters

Input:
X – the input data, a numpy array of shape (height*width , number_of_examples) 
Comment: since the input is in grayscale we only have height and width, otherwise it would have been height*width*3
Y – the “real” labels of the data, a vector of shape (num_of_classes, number of examples)
Layer_dims – a list containing the dimensions of each layer, including the input
batch_size – the number of examples in a single training batch.

Output:
parameters – the parameters learnt by the system during the training (the same parameters that were updated in the update_parameters function).
costs – the values of the cost function (calculated by the compute_cost function). One value is to be saved after each 100 training iterations (e.g. 3000 iterations -> 30 values).


In [134]:
def L_layer_model(X, Y, layers_dims, learning_rate, num_iterations, batch_size,verbose=False):
    # initialize network
    num_layers=layers_dims.shape[0]
    parameters=initialize_parameters(layers_dims,verbose=False)
    #print(model)
    # break X,Y into batches
    for itter in range(num_iterations):
      m=X.shape[1]
      for idx in range(0,m,batch_size):      
          X_batch = X[:,idx:min(m,idx+batch_size)]  
          Y_batch = Y[:,idx:min(m,idx+batch_size)] 
          AL,caches=L_model_forward(X_batch,parameters,num_layers,False,verbose=False)

          #print(caches)

          J=compute_cost(AL,Y_batch,verbose=False)

          

          if verbose:
                  print('itteration ',itter, 'index ',idx,'cost ',J)
                  print('len caches',len(caches))
                  for c in caches:
                    print('chache ------------------')
                    print(c)
                    #print(c[0],c[1]['A'])         
          grads=L_model_backward(AL, Y_batch, caches,num_layers,verbose=False)
          
          #print(grads)

          parameters=update_parameters(parameters, grads, learning_rate,num_layers,verbose=False)
          if verbose:
            print('running batch ',idx,' to ',idx+batch_size,'   Activation results  of last layer:')
            print('cost is ',J)
            
      print('itteration ',itter, 'index ',idx,'cost ',J)
      
      if verbose:
        for c in caches:
          print('Activations ------------------')
          print(c[0],c[1]['A'])
            
    return caches,parameters
    

In [131]:
def make_one_hot(y_in):     ### input is (m,)

  m=y_in.shape[0]
  y_t=y_in.reshape(m,1)
  classes = y_t.max()+1
  train_y = np.zeros((m, classes))
#print(train_y.shape)
#replacing 0 with a 1 at() the index of the original array
  rows=np.arange(m)
  train_y[rows,y_t] = 1
  return train_y.T      ### output  (classes,m)

b.	Predict(X, Y, parameters)
Description:
The function receives an input data and the true labels and calculates the accuracy of the trained neural network on the data.

Input:
X – the input data, a numpy array of shape (height*width, number_of_examples)
Y – the “real” labels of the data, a vector of shape (num_of_classes, number of examples)
Parameters – a python dictionary containing the DNN architecture’s parameters

Output:
accuracy – the accuracy measure of the neural net on the provided data (i.e. the percentage of the samples for which the correct label receives the hughest confidence score). Use the softmax function to normalize the output values.


In [67]:
def Predict(X, Y, parameters, number_of_layers,verbose=False):
        AL=X
        for l in range(1,number_of_layers-1):
            W=parameters['W'+str(l)]
            B=parameters['B'+str(l)]
            if verbose:
              print('layer ',l,' of ',number_of_layers)
              print(parameters['W'+str(l)].shape,parameters['B'+str(l)].shape)
            A=AL
            AL,c = linear_activation_forward(A,W,B,'relu')
            #caches.append(['Layer'+str(l),c])

        l=l+1   # for last layer  
        W=parameters['W'+str(l)]
        B=parameters['B'+str(l)]
        Y_pred,c= linear_activation_forward(AL,W,B,'softmax')

        a=np.argmax(Y_predict,axis=0)
        b=np.argmax(Y,axis=0)
        if verbose:
          print(b.shape)
          print(a,b)
        accuracy = np.sum(Y_pred == Y)/b.shape[0]

        #caches.append(['Layer'+str(l),c])
        return accuracy,Y_pred

In [57]:

##  testing training cycle 

l_dims=np.array([3,4,7,2])
number_of_layers = len(l_dims)
#print('num layers',number_of_layers)
print('neurons per layer',l_dims)

#
X=np.random.rand(3,20)
print('X - input ',X.shape)
Y=np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1,1, 1, 1,1, 1, 1],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
print('Y - output ',Y.shape)    # Y dimension should be (m = number of examples,n = output layer neurons)

#print(Y)
print("------------------------")
cache,parameters = L_layer_model(X,Y,l_dims,0.01,5,5,verbose=False)


with open('parameters.pickle', 'wb') as handle:
    pickle.dump(parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)

#print(cache)

neurons per layer [3 4 7 2]
X - input  (3, 20)
Y - output  (2, 20)
------------------------
itteration  0 index  0 cost  2.3025847742290195
itteration  0 index  5 cost  2.301585178546798
itteration  0 index  10 cost  2.300587003873983
itteration  0 index  15 cost  2.299589648354293
itteration  1 index  0 cost  2.2985939827496438
itteration  1 index  5 cost  2.2975991704774614
itteration  1 index  10 cost  2.296605801746204
itteration  1 index  15 cost  2.29561323384253
itteration  2 index  0 cost  2.2946223533541055
itteration  2 index  5 cost  2.293632315311992
itteration  2 index  10 cost  2.292643740729141
itteration  2 index  15 cost  2.2916559499565268
itteration  3 index  0 cost  2.290669844297424
itteration  3 index  5 cost  2.289684570820643
itteration  3 index  10 cost  2.2887007781224376
itteration  3 index  15 cost  2.2877177546872285
itteration  4 index  0 cost  2.286736413283035
itteration  4 index  5 cost  2.28575589460981
itteration  4 index  10 cost  2.2847768718943438


In [70]:
#  Running Prediction

with open('parameters.pickle', 'rb') as handle:
    parameters = pickle.load(handle)

l_dims=np.array([3,4,7,2])
number_of_layers = len(l_dims)
#print('num layers',number_of_layers)
print('neurons per layer',l_dims)

#
X=np.random.rand(3,20)
print('X - input ',X.shape)
Y=np.array([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1 ,1,1, 1, 1,1, 1, 1],[1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
print('Y - output ',Y.shape)    # Y dimension should be (m = number of examples,n = output layer neurons)

accuracy,Y_predict = Predict(X, Y, parameters, number_of_layers,verbose=True)

print('Accuracy is ',accuracy)

neurons per layer [3 4 7 2]
X - input  (3, 20)
Y - output  (2, 20)
layer  1  of  4
(4, 3) (4, 1)
layer  2  of  4
(7, 4) (7, 1)
(20,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Accuracy is  0.0


4.	Use the code you wrote to classify the MNIST dataset and present a summary report
a.	Load the dataset using the Keras code. Note that there is a predefined division between the train and test set. Use 20% of the training set as a validation set (samples need to be randomly chosen).
b.	Run your network using the following configuration:
•	4 layers (aside from the input layer), with the following sizes: 20,7,5,10
•	Do not activate the batchnorm option at this point
•	The input at each iteration needs to be “flattened” to a matrix of [m,784], where m is the number of samples
•	Use a learning rate of 0.009
•	Train the network until there is no improvement on the validation set (or the improvement is very small) for 100 training steps (this is the stopping criterion). Please include in the report the number of iterations and epochs needed to train your network. Also, specify the batch size.
c.	Please include the following details in your report:
•	The final accuracy values for the train, validation and test sets.
•	The cost value for each 100 training steps. Please make sure that the index of the training step will also be included in the report. Print the values from the L_layer_model
d.	All the information requested above will be included in a .docx file uploaded with the code.


In [135]:
##  MNIST training cycle 
from tensorflow import keras
from sklearn.model_selection import train_test_split

verbose=False
(x, y), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train, X_valid, y_train, y_valid = train_test_split( x, y, test_size=0.2, random_state=42)
if verbose:
  print(X_train.shape)
  print(X_valid.shape)
  print(X_test.shape)
  print(y_train.shape)
  print(y_valid.shape)
  print(y_test.shape)

l_dims=np.array([784,20,7,5,10])
number_of_layers = len(l_dims)
if verbose:
  #print('num layers',number_of_layers)
  print('neurons per layer',l_dims)

# 

train_x = X_train.reshape((-1, 784)).T

train_y=make_one_hot(y_train)

if verbose:
  print('X - input ',X.shape)
  print('Y - output ',Y.shape)    

#print(Y)
print("-----start train -------------------")
cache,parameters = L_layer_model(train_x,train_y,l_dims,0.1,10,50,verbose=False)


with open('parameters.pickle', 'wb') as handle:
    pickle.dump(parameters, handle, protocol=pickle.HIGHEST_PROTOCOL)

#print(cache)


-----start train -------------------
itteration  0 index  47950 cost  62.1460809842219
itteration  1 index  47950 cost  62.146080984221925
itteration  2 index  47950 cost  62.14608098422192
itteration  3 index  47950 cost  62.14608098422192
itteration  4 index  47950 cost  62.1460809842219
itteration  5 index  47950 cost  62.1460809842219
itteration  6 index  47950 cost  62.14608098422192
itteration  7 index  47950 cost  62.14608098422191
itteration  8 index  47950 cost  62.146080984221925
itteration  9 index  47950 cost  62.146080984221925


In [133]:
#  Running MNIST Prediction

with open('parameters.pickle', 'rb') as handle:
    parameters = pickle.load(handle)

#
X=X_valid.reshape((-1, 784)).T

Y=make_one_hot(y_valid)

accuracy,Y_predict = Predict(X, Y, parameters, 5 ,verbose=True)

print('Accuracy is ',accuracy)

layer  1  of  5
(20, 784) (20, 1)
layer  2  of  5
(7, 20) (7, 1)
layer  3  of  5
(5, 7) (5, 1)
(12000,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [0 0 0 ... 0 0 0]
Accuracy is  0.0


array([[0.02549902, 0.02549901, 0.025499  , 0.02549901, 0.02549899,
        0.02549901, 0.02549898, 0.025499  , 0.02549898, 0.025499  ,
        0.02549898, 0.02549899, 0.02549899, 0.02549897, 0.02549897,
        0.025499  , 0.02549902, 0.02549899, 0.025499  , 0.02549898],
       [0.02450101, 0.02450101, 0.02450102, 0.02450102, 0.024501  ,
        0.02450099, 0.02450101, 0.02450102, 0.02450101, 0.02450102,
        0.024501  , 0.024501  , 0.02450099, 0.024501  , 0.024501  ,
        0.02450101, 0.02450101, 0.024501  , 0.02450101, 0.024501  ]])

In [ ]:
####   Object Oriented Implementation of the Network

class Dense_Layer:
  def __init__(self,n_inputs,n_neurons):
    self.weights = np.random.randn(n_inputs,n_neurons)
    self.biases = np.zeros(n_neurons)
  def f_forward(self,inputs):
    self.outputs=np.dot(inputs,self.weights)+self.biases

X_train = [[1,2,3,4],[5,6,7,8],[9,9,9,9]]
Layer1 = Dense_Layer(4,5)
Layer2 = Dense_Layer(5,2) 

Layer1.f_forward(X_train)
Layer2.f_forward(Layer1.outputs)

print(Layer2.outputs)
